In [4]:
with open("test.dxf", "r") as f:
    data = f.read()
print(data)

  0
SECTION
  2
BLOCKS
  0
ENDSEC
  0
SECTION
  2
ENTITIES
  0
LINE
  8
0
 10
1.5
 20
0
 11
1.46722
 21
0.311868
  0
LINE
  8
0
 10
1.46722
 20
0.311868
 11
1.37032
 21
0.610105
  0
LINE
  8
0
 10
1.37032
 20
0.610105
 11
1.21353
 21
0.881678
  0
LINE
  8
0
 10
1.21353
 20
0.881678
 11
1.0037
 21
1.11472
  0
LINE
  8
0
 10
1.0037
 20
1.11472
 11
0.75
 21
1.29904
  0
LINE
  8
0
 10
0.75
 20
1.29904
 11
0.463525
 21
1.42658
  0
LINE
  8
0
 10
0.463525
 20
1.42658
 11
0.156793
 21
1.49178
  0
LINE
  8
0
 10
0.156793
 20
1.49178
 11
-0.156793
 21
1.49178
  0
LINE
  8
0
 10
-0.156793
 20
1.49178
 11
-0.463525
 21
1.42658
  0
LINE
  8
0
 10
-0.463525
 20
1.42658
 11
-0.75
 21
1.29904
  0
LINE
  8
0
 10
-0.75
 20
1.29904
 11
-1.0037
 21
1.11472
  0
LINE
  8
0
 10
-1.0037
 20
1.11472
 11
-1.21353
 21
0.881678
  0
LINE
  8
0
 10
-1.21353
 20
0.881678
 11
-1.37032
 21
0.610105
  0
LINE
  8
0
 10
-1.37032
 20
0.610105
 11
-1.46722
 21
0.311868
  0
LINE
  8
0
 10
-1.46722
 20
0.311868
 11
-1.5
 21